# IRMAS Test Evaluation

Load the trained `CNNVarTime` checkpoint, run it on the precomputed IRMAS test mel windows, aggregate predictions back to the clip level, and report clip-level accuracy.


In [13]:

from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader
import torch, numpy as np, pandas as pd
from collections import defaultdict
from src.models import CNNVarTime

PROJECT_ROOT = Path.cwd()
IRMAS_TEST_MANIFEST = PROJECT_ROOT / "data/manifests/irmas_test_mels.csv"
RESUME_CKPT = PROJECT_ROOT / "saved_weights/irmas_pretrain/best_val_acc.pt"
BATCH_SIZE = 32

In [14]:
# from src.utils.checkpoint import load_model_state  # ImportError: cannot import name 'load_model_state'
from src.utils.datasets import IRMASTestWindowDataset
# Select device (cuda if available, else cpu)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define class names in the same order as training
train_order_names = [
    "cel", "cla", "flu", "gac", "gel", "org", "pia", "sax", "tru", "vio", "voi"
]
test_dataset = IRMASTestWindowDataset(
    manifest_csv=IRMAS_TEST_MANIFEST,
    project_root=PROJECT_ROOT,
    class_names=train_order_names,
    per_example_norm=True,   # must match your train setting
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=None,
    collate_fn=None,
)

# Build model + strict load
model = CNNVarTime(num_classes=len(train_order_names)).to(DEVICE)
# state_dict = load_model_state(RESUME_CKPT)
# missing, unexpected = model.load_state_dict(state_dict, strict=True)
# assert not missing and not unexpected, f"state_dict mismatch: {missing} | {unexpected}"
w = model.state_dict()["conv1.weight"]
print("conv1 weight mean/std:", float(w.mean()), float(w.std()))
model.eval()


conv1 weight mean/std: 0.00943511351943016 0.10120747238397598


CNNVarTime(
  (conv1): Conv2d(2, 8, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (gap): AdaptiveAvgPool2d(outpu

In [16]:
logits_by_clip = defaultdict(list)
labels_by_clip = {}
windows_per_clip = defaultdict(int)
pin_memory = (DEVICE == "cuda")

with torch.no_grad():
    for batch in test_loader:
        # Skip batches with missing files
        if batch is None or any(item is None for item in batch):
            continue
        inputs, targets, clip_ids, paths = batch
        inputs = inputs.to(DEVICE, non_blocking=pin_memory)
        logits = model(inputs)                         # (B, C)
        logits_cpu = logits.detach().cpu().numpy()
        targets_np = targets.cpu().numpy()

        for logit_vec, target_vec, clip_id in zip(logits_cpu, targets_np, clip_ids):
            logits_by_clip[clip_id].append(logit_vec)
            labels_by_clip[clip_id] = target_vec.astype(np.float32)
            windows_per_clip[clip_id] += 1

# --- aggregate per clip & score single-label membership ---
rows = []
for clip_id, logit_list in logits_by_clip.items():
    stacked = np.stack(logit_list, 0)                # (W, C)
    mean_logits = stacked.mean(axis=0)               # (C,)
    exp = np.exp(mean_logits - mean_logits.max())
    probs = exp / exp.sum()
    pred_idx = int(mean_logits.argmax())
    gt_vec = labels_by_clip[clip_id]
    hit1 = bool(gt_vec[pred_idx] > 0.5)

    order = np.argsort(mean_logits)[::-1]
    top3 = ", ".join(f"{train_order_names[i]} ({probs[i]:.2f})" for i in order[:3])
    true_names = [train_order_names[i] for i, v in enumerate(gt_vec) if v > 0.5]

    rows.append({
        "clip": clip_id,
        "true_labels": "|".join(true_names),
        "pred_label": train_order_names[pred_idx],
        "pred_score": float(probs[pred_idx]),
        "top3": top3,
        "hit@1": hit1,
        "windows": int(windows_per_clip[clip_id]),
    })

df = pd.DataFrame(rows).sort_values(["hit@1", "pred_score"], ascending=[True, False])
print("Top-1 clip accuracy:", df["hit@1"].mean())
df.head()

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 3.
Original Traceback (most recent call last):
  File "C:\Users\hu_ju\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\_utils\worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hu_ju\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\_utils\fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hu_ju\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\_utils\fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for id

In [ ]:
per_class = []
for ci, cname in enumerate(train_order_names):
    mask = df["true_labels"].str.contains(rf"(?:^|[|]){cname}(?:$|[|])")
    support = int(mask.sum())
    correct = int((df.loc[mask, "pred_label"] == cname).sum())
    per_class.append({"class": cname, "support": support, "hit_rate": (correct / support if support else float("nan"))})
per_class_df = pd.DataFrame(per_class).sort_values("hit_rate", ascending=False)
per_class_df
